### Lesson 6


# Sharpe Ratio based Portfolio Optimization 

![](Lesson6GoalHeaderImage.png)

## 6.1  Introduction

**Sharpe Ratio**, developed by Nobel Laureate William F Sharpe [SHA 66], is a measure of calculating risk adjusted return. It serves to help investors know about the returns on their investments relative to the risks they hold. The Sharpe Ratio is defined as

![](Lesson6Eqn6_1.png)
<h5 align="right">..........(6.1)</h5>


where $r_x$ is the average rate of return on the investment $x$, $R_f$,  the best available **risk free rate of return** and $\sigma$ the standard deviation of $r_x$, which denotes the risk on the investment. 

Higher the Sharpe Ratio, more is the excess returns over that of holding a risk free investment, relative to the increased volatility that the investment is exposed to. A Sharpe Ratio of 0, needless to say,  only denotes the investment to be risk-free or one that does not yield any excess return. In practice, while a Sharpe ratio of 1 marks the investment to be acceptable or good for investors, a value less than 1 grades the investment as  sub-optimal,  and values greater than 1 and moving towards 2 or 3, grades the investment as  highly superior. 

## 6.2 Maximizing Sharpe Ratio

Having understood the significance of the Sharpe Ratio, let us suppose an investor wishes to make an investment in  assets in such a way that the Sharpe Ratio of the portfolio would be the best possible or the maximum, that can be ensured for the investment.  


Let P be a portfolio comprising assets $A_1, A_2, ...A_N$, with $\mu_1, \mu_2, ...\mu_N$ as the asset returns and $W_1, W_2, ...W_N$ as the weights. 
 

The portfolio return $r$ determined by a weighted summation of its individual asset returns is given by, $\sum\left({W_i.\mu_i}\right)$ and the risk is given by $\sqrt{\sum\sum {W_i.W_j.\sigma_{ij}} } $. (See **Lesson 1 Fundamentals of Risk and Return of a Portfolio** to know about risk and return of a portfolio).  

To keep the discussion simple for now, let us suppose that  the investor decides to enforce only **basic constraints** on the portfolio. (See Sec. 5.2 of **Lesson 5 Mean Variance Optimization of Portfolios** to know about basic constraints).  
  
The mathematical model for the Sharpe Ratio based Portfolio Optimization is given by, 

![](Lesson6Eqn6_2.png)
<h5 align="right">..........(6.2)</h5>

The numerator of the objective function denotes the excess returns of the investment over that of a risk free asset $R_f$ and the denominator the risk of the investment. The objective is to maximize the Sharpe Ratio. The basic constraints indicate that the investor wishes to have a fully invested portfolio. 

## 6.3 Solving the Sharpe Ratio Optimization Model

To solve the Sharpe Ratio maximization model represented by (6.2), we make use of the **minimize** library function from **scipy.optimize** package of Python. However, since the original objective function insists on maximization as opposed to minimization demanded by the **minimize** solver, the **principal of  duality** borrowed from Optimization Theory is employed to undertake the transformation. According to the principle,  


![](Lesson6Eqn6_3.png)
<h5 align="right">..........(6.3)</h5>

The Python code for the function **MaximizeSharpeRatioOptimization** which defines the objective function  and the basic constraints represented by (6.2),  is shown below: 

In [231]:
#function to undertake Sharpe Ratio maximization subject to 
#basic constraints of the portfolio

#dependencies
import numpy as np
from scipy import optimize 
import random

def MaximizeSharpeRatioOptmzn(MeanReturns, CovarReturns, RiskFreeRate, PortfolioSize):
    
    # define maximization of Sharpe Ratio using principle of duality
    def  f(x, MeanReturns, CovarReturns, RiskFreeRate, PortfolioSize):
        funcDenomr = np.sqrt(np.matmul(np.matmul(x, CovarReturns), x.T) )
        funcNumer = np.matmul(np.array(MeanReturns),x.T)-RiskFreeRate
        func = -(funcNumer / funcDenomr)
        return func

    #define equality constraint representing fully invested portfolio
    def constraintEq(x):
        A=np.ones(x.shape)
        b=1
        constraintVal = np.matmul(A,x.T)-b 
        return constraintVal
    
    
    #define bounds and other parameters
    xinit=np.repeat(0.33, PortfolioSize)
    cons = ({'type': 'eq', 'fun':constraintEq})
    lb = 0
    ub = 1
    bnds = tuple([(lb,ub) for x in xinit])
    
    #invoke minimize solver
    opt = optimize.minimize (f, x0 = xinit, args = (MeanReturns, CovarReturns,\
                             RiskFreeRate, PortfolioSize), method = 'SLSQP',  \
                             bounds = bnds, constraints = cons, tol = 10**-3)
    
    return opt
    

The Sharpe Ratio optimization requires the computation of risk and return of the portfolio. The asset returns computing function **StockReturnsComputing**, is reproduced here for the reader's convenience. 

In [232]:
# function computes asset returns 
def StockReturnsComputing(StockPrice, Rows, Columns):
    
    import numpy as np
    
    StockReturn = np.zeros([Rows-1, Columns])
    for j in range(Columns):        # j: Assets
        for i in range(Rows-1):     # i: Daily Prices
            StockReturn[i,j]=((StockPrice[i+1, j]-StockPrice[i,j])/StockPrice[i,j])* 100

    return StockReturn

## 6.4  Case Study

 Let us suppose that an investor decides to invest in a $k$-portfolio ($k$-portfolio 1) comprising the  following  Dow stocks. ($k$ portfolio 1, is  detailed and listed in **Lesson 3 Heuristic Portfolio Selection**)  

**𝑘-portfolio 1**:  

{Coca-Cola (KO), United Health (UNH), Walt Disney (DIS), IBM (IBM), Cisco (CSCO), JPMorgan Chase (JPM), Goldman Sachs (GS), Walgreens Boots Alliance (WBA), Apple (AAPL), Home Depot (HD), American Express (AXP), McDonald's (MCD), Merck (MRK), Boeing (BA), Caterpillar (CAT)}       

The investor desires to explore the optimal portfolio set that would yield the maximal Sharpe Ratio.  The objective is to find out  the optimal weights that will ensure maximal Sharpe Ratio for the portfolio.

The following Python code reads the dataset concerned, computes the stock returns using the Python function **StockReturnsComputing** and obtains the mean returns and the variance-covariance matrix of returns. 


In [233]:
#obtain mean and variance-covariance matrix of returns for k-portfolio 1

#Dependencies
import numpy as np
import pandas as pd

## meanreturns = 
df_ESG = pd.read_csv('ESGSCORES.csv')

ESGScores = pd.DataFrame(df_ESG,columns=['Symbol','ESG Score'])

arr_ESGScores=ESGScores.to_numpy().T
print(arr_ESGScores)
ESGScores = pd.DataFrame(arr_ESGScores)
ESGScores.columns = ESGScores.iloc[0]
ESGScores = ESGScores.iloc[1: , :]
ESGScores.head()

[['SPY' 'IVV' 'VTI' ... 'GBGR' 'AILV' 'AILG']
 [8.06 8.06 7.68 ... 5.38 7.28 5.68]]


,SPY,IVV,VTI,VOO,QQQ,VTV,AGG,BND,VUG,IJR,...,EEH,SKYU,STLG,HHH,AGGH,DSPC,EPRE,GBGR,AILV,AILG
1,8.06,8.06,7.68,8.06,7.99,7.7,6.51,6.63,8.23,5.26,...,0.0,0.0,8.85,5.18,0.0,0.0,6.94,5.38,7.28,5.68


In [ ]:
#input k portfolio 1 dataset comprising 15 stocks
Rows = 1259  #excluding header
Columns = 10 #excluding date

#read stock prices 
df = pd.read_csv('all_stocks.csv')
df2 = pd.read_csv('data.csv')

In [ ]:
df2["fund_symbol"].unique()

In [ ]:
df.head()

In [ ]:
df3= df.loc[:, df.columns != 'price_date']
choice=df3.columns.tolist()
tickers=[]
for x in range(0,11):
    notfound=True
    while(notfound == True):
        trial = random.choices(choice)[0]
        if (trial in df)== True:
            if (trial in tickers)==False:
                notfound=False
                tickers.append(trial)
print(tickers)
#extract asset labels
df=df[tickers]
assetLabels = df.columns[1:Columns+1].tolist()
print('Asset labels of k-portfolio 1: \n', assetLabels)


#extract ESG Scores
ESG=ESGScores[assetLabels]
esgscores=ESG.iloc[0].tolist()
print('ESG Scores of k-portfolio 1: \n',esgscores)

#read asset prices data
StockData = df.iloc[0:, 1:]


#compute asset returns
arStockPrices = np.asarray(StockData)
[Rows, Cols]=arStockPrices.shape
arReturns = StockReturnsComputing(arStockPrices, Rows, Cols)

#set precision for printing results
np.set_printoptions(precision=3, suppress = True)

#compute mean returns and variance covariance matrix of returns
meanReturns = np.mean(arReturns, axis = 0)
covReturns = np.cov(arReturns, rowvar=False)
print('\nMean Returns:\n', meanReturns)
print('\nVariance-Covariance Matrix of Returns:\n', covReturns)


In [ ]:
df.head()

The annual average risk free rate of return in USA during April 2019 was 3%. The daily risk free rate is computed  as   


![](Lesson6Eqn6_4.png)
<h5 align="right">..........(6.4)</h5>


The following Python code computes the maximal Sharpe Ratio for  $k$-portfolio 1.

In [ ]:
#obtain maximal Sharpe Ratio for k-portfolio 1 of Dow stocks

#set portfolio size
portfolioSize = Columns

#set risk free asset rate of return
Rf=3  # April 2019 average risk  free rate of return in USA approx 3%
annRiskFreeRate = Rf/100

#compute daily risk free rate in percentage
r0 = (np.power((1 + annRiskFreeRate),  (1.0 / 360.0)) - 1.0) * 100 
print('\nRisk free rate (daily %): ', end="")
print ("{0:.3f}".format(r0)) 

#initialization
xOptimal =[]
minRiskPoint = []
expPortfolioReturnPoint =[]
maxSharpeRatio = 0

#compute maximal Sharpe Ratio and optimal weights
result = MaximizeSharpeRatioOptmzn(meanReturns, covReturns, r0, portfolioSize)
xOptimal.append(result.x)

    
#compute risk returns and max Sharpe Ratio of the optimal portfolio  

xOptimalArray = np.array(xOptimal)
Risk = np.matmul((np.matmul(xOptimalArray,covReturns)), np.transpose(xOptimalArray))
expReturn = np.matmul(np.array(meanReturns),xOptimalArray.T)
annRisk =   np.sqrt(Risk*251) 
annRet = 251*np.array(expReturn) 
maxSharpeRatio = (annRet-Rf)/annRisk 
avesg=np.matmul(np.array(esgscores),xOptimalArray.T)
#set precision for printing results
np.set_printoptions(precision=3, suppress = True)


#display results
print('Maximal Sharpe Ratio: ', maxSharpeRatio, '\nAnnualized Risk (%):  ', \
      annRisk, '\nAnnualized Expected Portfolio Return(%):  ', annRet,'\nAverage porftilio ESG(%):  ', avesg)
print('\nOptimal weights (%):\n',  xOptimalArray.T*100 )

In [ ]:
#function to undertake Sharpe Ratio maximization subject to 
#basic constraints of the portfolio

#dependencies
import numpy as np
from scipy import optimize 
import random

def ESGMaximizeSharpeRatioOptmzn(ESG,MeanReturns, CovarReturns, RiskFreeRate, PortfolioSize):
    
    # define maximization of Sharpe Ratio using principle of duality
    def  f(x, ESG,MeanReturns, CovarReturns, RiskFreeRate, PortfolioSize):
        funcDenomr = np.sqrt(np.matmul(np.matmul(x, CovarReturns), x.T) )
        funcNumer = np.matmul(np.array(MeanReturns),x.T)-RiskFreeRate
        funcESG=np.matmul(np.array(ESG),x.T)
        func = -(funcESG)*(funcNumer / funcDenomr)
        return func

    #define equality constraint representing fully invested portfolio
    def constraintEq(x):
        A=np.ones(x.shape)
        b=1
        constraintVal = np.matmul(A,x.T)-b 
        return constraintVal
    
    
    #define bounds and other parameters
    xinit=np.repeat(0.33, PortfolioSize)
    cons = ({'type': 'eq', 'fun':constraintEq})
    lb = 0
    ub = 1
    bnds = tuple([(lb,ub) for x in xinit])
    
    #invoke minimize solver
    opt = optimize.minimize (f, x0 = xinit, args = (ESG,MeanReturns, CovarReturns,\
                             RiskFreeRate, PortfolioSize), method = 'SLSQP',  \
                             bounds = bnds, constraints = cons, tol = 10**-3)
    
    return opt
    

In [ ]:
#obtain maximal Sharpe Ratio for k-portfolio 1 of Dow stocks

#set portfolio size
portfolioSize = Columns

#set risk free asset rate of return
Rf=3  # April 2019 average risk  free rate of return in USA approx 3%
annRiskFreeRate = Rf/100

#compute daily risk free rate in percentage
r0 = (np.power((1 + annRiskFreeRate),  (1.0 / 360.0)) - 1.0) * 100 
print('\nRisk free rate (daily %): ', end="")
print ("{0:.3f}".format(r0)) 

#initialization
xOptimal =[]
minRiskPoint = []
expPortfolioReturnPoint =[]
maxSharpeRatio = 0

#compute maximal Sharpe Ratio and optimal weights
result = ESGMaximizeSharpeRatioOptmzn(esgscores,meanReturns, covReturns, r0, portfolioSize)
xOptimal.append(result.x)

    
#compute risk returns and max Sharpe Ratio of the optimal portfolio   

xOptimalArray = np.array(xOptimal)
Risk = np.matmul((np.matmul(xOptimalArray,covReturns)), np.transpose(xOptimalArray))
expReturn = np.matmul(np.array(meanReturns),xOptimalArray.T)
annRisk =   np.sqrt(Risk*251) 
annRet = 251*np.array(expReturn) 
maxSharpeRatio = (annRet-Rf)/annRisk 
avesg=np.matmul(np.array(esgscores),xOptimalArray.T)
#set precision for printing results
np.set_printoptions(precision=3, suppress = True)


#display results
print('Maximal Sharpe Ratio: ', maxSharpeRatio, '\nAnnualized Risk (%):  ', \
      annRisk, '\nAnnualized Expected Portfolio Return(%):  ', annRet,'\nAverage porftilio ESG(%):  ', avesg)
print('\nOptimal weights (%):\n',  xOptimalArray.T*100 )

<h3 align="left">References</h3>   
 
[SHA 66]  Sharpe, William F. Mutual Fund Performance,  *Journal of Business*, January 1966, pp. 119-138.   

[PAI 18]   Vijayalakshmi Pai G. A., Metaheuristics for Portfolio Optimization- An Introduction using MATLAB, Wiley-ISTE, 2018. https://www.mathworks.com/academia/books/metaheuristics-for-portfolio-optimization-pai.html  


#### Next.....Lesson 7: Constrained Portfolio Optimization